# Leer Excels FORECAST INBOUND más recientes según semana

Este código:
- Identifica carpetas con formato `YYYY-MM` y las ordena por fecha.
- Según la variable `semana`:
  - **Semana 1 o 2**: Lee el Excel más reciente de la carpeta más reciente.
  - **Semana 3 o 4**: Lee el Excel más reciente de la carpeta más reciente (`df1`) y el de la penúltima carpeta más reciente (`df2`).

In [89]:
import os
import polars as pl
from datetime import datetime

ruta = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound")

semana = 3
tienda = 0
faltan = 1

carpetas = [carpeta for carpeta in os.listdir(ruta) if os.path.isdir(os.path.join(ruta, carpeta)) and len(carpeta) >= 7 and carpeta[:4].isdigit() and carpeta[5:7].isdigit()]
carpetas.sort(reverse=True, key=lambda x: datetime.strptime(x[:7], "%Y-%m"))

def obtener_excel_mas_reciente(carpeta_path):
    archivos = [f for f in os.listdir(carpeta_path) if f.lower().endswith('.xlsx')]
    archivos.sort(reverse=True, key=lambda x: os.path.getmtime(os.path.join(carpeta_path, x)))
    return os.path.join(carpeta_path, archivos[0]) if archivos else None

def leer_excel_desde_fila_4(ruta_archivo, hoja):
    df = pl.read_excel(
        ruta_archivo, 
        sheet_name=hoja, 
        read_options={"skip_rows": 1}
    )
    return df

df1 = None
if semana in [1, 2]:
    carpeta_mas_reciente = os.path.join(ruta, carpetas[0])
    print(f"Carpeta más reciente para semana {semana}: {carpeta_mas_reciente}")
    archivo_reciente = obtener_excel_mas_reciente(carpeta_mas_reciente)
    print(f"Archivo más reciente: {archivo_reciente}")
    if archivo_reciente:
        df1 = leer_excel_desde_fila_4(archivo_reciente, hoja="BASE")
elif semana in [3, 4]:
    carpeta_mas_reciente = os.path.join(ruta, carpetas[0])
    print(f"Carpeta más reciente para semana {semana}: {carpeta_mas_reciente}")
    archivo_reciente = obtener_excel_mas_reciente(carpeta_mas_reciente)
    print(f"Archivo más reciente: {archivo_reciente}")
    if archivo_reciente:
        df1 = leer_excel_desde_fila_4(archivo_reciente, hoja="BASE")
    carpeta_penultima = os.path.join(ruta, carpetas[1])
    print(f"Penúltima carpeta: {carpeta_penultima}")
    archivo_penultimo = obtener_excel_mas_reciente(carpeta_penultima)
    print(f"Archivo más reciente de la penúltima carpeta: {archivo_penultimo}")
    if archivo_penultimo:
        df2 = leer_excel_desde_fila_4(archivo_penultimo, hoja="BASE")


Carpeta más reciente para semana 3: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound\2024-11 Noviembre
Archivo más reciente: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound\2024-11 Noviembre\11.2024 S&OP Demanda Sin restricciones Inbound Ciclo Diciembre 24_Inbound.xlsx
Penúltima carpeta: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound\2024-10 Octubre
Archivo más reciente de la penúltima carpeta: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound\2024-10 Octubre\10.2024 S&OP Demanda Sin restricciones Inbound Ciclo Noviembre 24_Inbound.xlsx


### Descripción del Código

El código busca dinámicamente una fila donde la primera columna sea "ID" en los DataFrames `df1` y `df2`. Una vez encontrada:
1. Usa esa fila como encabezados de las columnas.
2. Elimina la fila utilizada como encabezado del DataFrame.
3. Renombra columnas duplicadas añadiendo sufijos únicos (`_1`, `_2`, etc.).

Esto asegura que los DataFrames tengan columnas únicas y estén listos para análisis.


In [90]:
if 'df1' in locals() and df1 is not None:
    for i, row in enumerate(df1.iter_rows(named=False)):
        if row[0] == "ID":
            nuevos_nombres = [str(col) for col in row]
            seen = {}
            for idx, col in enumerate(nuevos_nombres):
                if col in seen:
                    seen[col] += 1
                    nuevos_nombres[idx] = f"{col}_{seen[col]}"
                else:
                    seen[col] = 0
            df1 = df1.slice(i + 1)
            df1 = df1.rename({df1.columns[j]: nuevos_nombres[j] for j in range(len(nuevos_nombres))})
            break

if 'df2' in locals() and df2 is not None:
    for i, row in enumerate(df2.iter_rows(named=False)):
        if row[0] == "ID":
            nuevos_nombres = [str(col) for col in row]
            seen = {}
            for idx, col in enumerate(nuevos_nombres):
                if col in seen:
                    seen[col] += 1
                    nuevos_nombres[idx] = f"{col}_{seen[col]}"
                else:
                    seen[col] = 0
            df2 = df2.slice(i + 1)
            df2 = df2.rename({df2.columns[j]: nuevos_nombres[j] for j in range(len(nuevos_nombres))})
            break

# AISLAR FORECAST FINAL

In [91]:
df_final_1 = None
df_final_2 = None

if 'df1' in locals() and df1 is not None:
    columnas_seleccionadas = [0, 1] + list(range(170, 182))
    df_final_1 = df1.select([df1.columns[i] for i in columnas_seleccionadas])

if 'df2' in locals() and df2 is not None:
    columnas_seleccionadas = [0, 1] + list(range(170, 182))
    df_final_2 = df2.select([df2.columns[i] for i in columnas_seleccionadas])

### DF FORECAST FINAL

El código realiza lo siguiente:

1. Si `df_final_1` o `df_final_2` existen:
   - Mantiene las primeras dos columnas.
   - Realiza un `melt` en las demás columnas para convertirlas en formato largo.
   - Trunca los valores de la columna `variable` (generada por el `melt`) a los primeros 10 caracteres.

2. Resultado:
   - `df_melt_1`: Versión procesada de `df_final_1`.
   - `df_melt_2`: Versión procesada de `df_final_2`.


In [92]:
df_melt_1 = None
df_melt_2 = None

if 'df_final_1' in locals() and df_final_1 is not None:
    df_melt_1 = df_final_1.unpivot(
        index=df_final_1.columns[:2],
        on=df_final_1.columns[2:]
    )
    df_melt_1 = df_melt_1.with_columns(
        df_melt_1['variable'].str.slice(0, 10).alias('variable')
    )

if 'df_final_2' in locals() and df_final_2 is not None:
    df_melt_2 = df_final_2.unpivot(
        index=df_final_2.columns[:2],
        on=df_final_2.columns[2:]
    )
    df_melt_2 = df_melt_2.with_columns(
        df_melt_2['variable'].str.slice(0, 10).alias('variable')
    )


pasara variable a formato fecha

In [93]:
if 'df_melt_1' in locals() and df_melt_1 is not None:
    df_melt_1 = df_melt_1.with_columns(
        pl.col('variable').str.strptime(pl.Date, "%Y-%m-%d").alias('variable')
    )

if 'df_melt_2' in locals() and df_melt_2 is not None:
    df_melt_2 = df_melt_2.with_columns(
        pl.col('variable').str.strptime(pl.Date, "%Y-%m-%d").alias('variable')
    )


crear columna fc semanal que es = a FC / 4,33

In [94]:
if 'df_melt_1' in locals() and df_melt_1 is not None:
    df_melt_1 = df_melt_1.with_columns(
        pl.col('value').cast(pl.Float64).alias('value')
    ).with_columns(
        (pl.col('value') / 4.33).alias('Fc Semanal')
    )

if 'df_melt_2' in locals() and df_melt_2 is not None:
    df_melt_2 = df_melt_2.with_columns(
        pl.col('value').cast(pl.Float64).alias('value')
    ).with_columns(
        (pl.col('value') / 4.33).alias('Fc Semanal')
    )


## SI ES SEMANA 1 Y 2 NO JUNTAR FC PERO SI ES SEMANA 3 O 4 ENTONCES SI JUNTARLOS

### Filtrar `df_melt_2` por la fecha más antigua en la columna "variable"

El siguiente bloque de código verifica si `df_melt_2` existe y no es `None`. Si cumple con estas condiciones, filtra el dataframe dejando solo los registros correspondientes a la fecha más antigua en la columna `variable`:

In [95]:
if 'df_melt_2' in locals() and df_melt_2 is not None:
    df_melt_2 = df_melt_2.filter(
        pl.col('variable') == df_melt_2.select(pl.col('variable').min())[0, 0]
    )

### Anexar `df_melt_2` a `df_melt_1` y ordenar por la columna "variable"

Este bloque de código verifica si `df_melt_2` existe y no es `None`. Si cumple con estas condiciones, agrega los datos de `df_melt_2` a `df_melt_1` y luego ordena el dataframe resultante por la columna `variable` en orden ascendente:

In [96]:
print(df_melt_1[0])

shape: (1, 5)
┌──────────────────┬──────────────┬────────────┬───────┬────────────┐
│ ID               ┆ Ult. Eslabón ┆ variable   ┆ value ┆ Fc Semanal │
│ ---              ┆ ---          ┆ ---        ┆ ---   ┆ ---        │
│ str              ┆ str          ┆ date       ┆ f64   ┆ f64        │
╞══════════════════╪══════════════╪════════════╪═══════╪════════════╡
│ 1171331CL RETAIL ┆ 1171331      ┆ 2024-12-01 ┆ 810.0 ┆ 187.066975 │
└──────────────────┴──────────────┴────────────┴───────┴────────────┘


In [97]:
print(df_melt_1[0])

shape: (1, 5)
┌──────────────────┬──────────────┬────────────┬───────┬────────────┐
│ ID               ┆ Ult. Eslabón ┆ variable   ┆ value ┆ Fc Semanal │
│ ---              ┆ ---          ┆ ---        ┆ ---   ┆ ---        │
│ str              ┆ str          ┆ date       ┆ f64   ┆ f64        │
╞══════════════════╪══════════════╪════════════╪═══════╪════════════╡
│ 1171331CL RETAIL ┆ 1171331      ┆ 2024-12-01 ┆ 810.0 ┆ 187.066975 │
└──────────────────┴──────────────┴────────────┴───────┴────────────┘


In [98]:
if 'df_melt_2' in locals() and df_melt_2 is not None:
    df_melt_1 = df_melt_1.vstack(df_melt_2).sort('variable')
print(df_melt_1[0])


shape: (1, 5)
┌──────────────────┬──────────────┬────────────┬───────┬────────────┐
│ ID               ┆ Ult. Eslabón ┆ variable   ┆ value ┆ Fc Semanal │
│ ---              ┆ ---          ┆ ---        ┆ ---   ┆ ---        │
│ str              ┆ str          ┆ date       ┆ f64   ┆ f64        │
╞══════════════════╪══════════════╪════════════╪═══════╪════════════╡
│ 1000078CL RETAIL ┆ 1000078      ┆ 2024-11-01 ┆ 0.0   ┆ 0.0        │
└──────────────────┴──────────────┴────────────┴───────┴────────────┘


### LECTURA DEL TUBO

1. Filtra carpetas que contengan `-` en su nombre y evalúa si tienen fechas en formato `YYYY-MM-DD`.
2. Selecciona la carpeta con la fecha más reciente.
3. Dentro de esa carpeta:
   - Busca el archivo `.xlsx` que contenga `stock` y `s4`.
   - Busca el archivo `.xlsx` que contenga `stock` y `tiendas`.
   - Busca el archivo `.xlsx` que contenga `tr final s4 consolidado`.
4. Carga los archivos como DataFrames (`df_stock`, `df_tiendas`, `df_transito`) considerando solo la hoja `Sheet1`.


In [99]:
import os
import polars as pl
from datetime import datetime

ruta_principal = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Tubo Semanal")


carpetas = [
    carpeta for carpeta in os.listdir(ruta_principal)
    if '-' in carpeta and carpeta[:10].replace('-', '').isdigit()
]

fechas_carpetas = [
    (carpeta, datetime.strptime(carpeta[:10], "%Y-%m-%d"))
    for carpeta in carpetas
]

if fechas_carpetas:
    carpeta_reciente = max(fechas_carpetas, key=lambda x: x[1])[0]
    ruta_carpeta_reciente = os.path.join(ruta_principal, carpeta_reciente)

    archivos = [
        archivo for archivo in os.listdir(ruta_carpeta_reciente)
        if archivo.lower().endswith('.xlsx')
    ]

    archivo_stock_s4 = next(
        (os.path.join(ruta_carpeta_reciente, archivo)
         for archivo in archivos if 'stock' in archivo.lower() and 's4' in archivo.lower()), None
    )
    archivo_stock_tiendas = next(
        (os.path.join(ruta_carpeta_reciente, archivo)
         for archivo in archivos if 'stock' in archivo.lower() and 'tiendas' in archivo.lower()), None
    )
    archivo_tr_final_s4 = next(
        (os.path.join(ruta_carpeta_reciente, archivo)
         for archivo in archivos if 'tr final s4 consolidado' in archivo.lower()), None
    )

    df_stock = pl.read_excel(archivo_stock_s4, sheet_name="Sheet1") if archivo_stock_s4 else None
    df_tiendas = pl.read_excel(archivo_stock_tiendas, sheet_name="Sheet1") if archivo_stock_tiendas else None
    df_transito = pl.read_excel(archivo_tr_final_s4, sheet_name="Sheet1") if archivo_tr_final_s4 else None

    print("Carpeta más reciente:", ruta_carpeta_reciente)
    print("Archivo Stock S4:", archivo_stock_s4)
    print("Archivo Stock Tiendas:", archivo_stock_tiendas)
    print("Archivo TR Final S4:", archivo_tr_final_s4)
else:
    print("No se encontraron carpetas válidas.")


Carpeta más reciente: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Tubo Semanal\2024-11-18
Archivo Stock S4: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Tubo Semanal\2024-11-18\2024-11-18 Stock S4.XLSX
Archivo Stock Tiendas: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Tubo Semanal\2024-11-18\2024-11-18 Stock Tiendas.XLSX
Archivo TR Final S4: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Tubo Semanal\2024-11-18\2024-11-18 TR FINAL S4 Consolidado.xlsx


# LIMPIEZA DE STOCK SEPARACION 711 DE LOS DEMAS, SUMA DE CONTROL

Este script organiza y agrupa datos de inventario de materiales en un DataFrame usando Polars. 

### Pasos Principales:

1. **Agrupación Inicial (`df_stock_agrupado`)**  
   Agrupa los datos por las columnas: `Material`, `Centro`, `Texto breve de material` y `Ult Eslabon`. Calcula la suma de las cantidades en las columnas:
   - `Libre utilización`
   - `Trans./Trasl.`
   - `Inspecc.de calidad`
   - `Bloqueado`

   Luego, se crea una nueva columna `Control`, que suma las columnas `Trans./Trasl.`, `Inspecc.de calidad` y `Bloqueado`.

2. **Separación por Centro (`711` y Otros)**  
   - Se filtra y elimina la columna `Centro` para generar dos DataFrames: 
     - `df_711`: Contiene solo datos del centro `711`.
     - `df_stock`: Contiene los demás centros.

3. **Reagrupación Final**  
   - `df_711`: Se reagrupan los datos por `Material`, `Texto breve de material` y `Ult Eslabon`, recalculando las sumas de las mismas columnas originales.
   - `df_stock`: Se reagrupa con las mismas columnas que `df_711`, pero incluyendo la suma de la columna adicional `Control`.

### Resultado:
Dos DataFrames (`df_711` y `df_stock`) organizados y listos para análisis, con las cantidades agregadas por material y eslabón. 


In [100]:
df_stock_agrupado = df_stock.group_by(['Material', 'Centro', 'Texto breve de material', 'Ult. Eslabon']).agg(
    [
        pl.col('Libre utilización').sum().alias('Libre utilización'),
        pl.col('Trans./Trasl.').sum().alias('Trans./Trasl.'),
        pl.col('Inspecc.de calidad').sum().alias('Inspecc.de calidad'),
        pl.col('Bloqueado').sum().alias('Bloqueado'),
    ]
)
df_stock_agrupado = df_stock_agrupado.with_columns(
    (
        pl.col('Trans./Trasl.') + pl.col('Inspecc.de calidad') + pl.col('Bloqueado')
    ).alias('Control')
)
df_711 = df_stock_agrupado.filter(pl.col('Centro').str.contains('711')).drop('Centro')
df_stock = df_stock_agrupado.filter(~pl.col('Centro').str.contains('711')).drop('Centro')

df_711 = df_711.group_by(['Material', 'Texto breve de material', 'Ult. Eslabon']).agg(
    [
        pl.col('Libre utilización').sum().alias('Libre utilización'),
        pl.col('Trans./Trasl.').sum().alias('Trans./Trasl.'),
        pl.col('Inspecc.de calidad').sum().alias('Inspecc.de calidad'),
        pl.col('Bloqueado').sum().alias('Bloqueado'),
    ]
)

df_stock = df_stock.group_by(['Material', 'Texto breve de material', 'Ult. Eslabon']).agg(
    [
        pl.col('Libre utilización').sum().alias('Libre utilización'),
        pl.col('Control').sum().alias('Control'),
        pl.col('Trans./Trasl.').sum().alias('Trans./Trasl.'),
        pl.col('Inspecc.de calidad').sum().alias('Inspecc.de calidad'),
        pl.col('Bloqueado').sum().alias('Bloqueado'),
    ]
)

In [101]:
df_711 = df_711.rename({col: 'UE' for col in df_711.columns if 'labon' in col})
print(df_711[0])

shape: (1, 7)
┌──────────┬───────────────────────┬────────┬─────────────┬───────────────┬────────────┬───────────┐
│ Material ┆ Texto breve de        ┆ UE     ┆ Libre       ┆ Trans./Trasl. ┆ Inspecc.de ┆ Bloqueado │
│ ---      ┆ material              ┆ ---    ┆ utilización ┆ ---           ┆ calidad    ┆ ---       │
│ str      ┆ ---                   ┆ str    ┆ ---         ┆ i64           ┆ ---        ┆ f64       │
│          ┆ str                   ┆        ┆ f64         ┆               ┆ i64        ┆           │
╞══════════╪═══════════════════════╪════════╪═════════════╪═══════════════╪════════════╪═══════════╡
│ 130730   ┆ FILTRO AIRE INT. NH   ┆ 130730 ┆ 6.0         ┆ 0             ┆ 0          ┆ 0.0       │
└──────────┴───────────────────────┴────────┴─────────────┴───────────────┴────────────┴───────────┘


In [102]:
df_stock = df_stock.rename({col: 'UE' for col in df_stock.columns if 'labon' in col})
print(df_stock[0])

shape: (1, 8)
┌──────────┬──────────────┬────────┬─────────────┬─────────┬─────────────┬─────────────┬───────────┐
│ Material ┆ Texto breve  ┆ UE     ┆ Libre       ┆ Control ┆ Trans./Tras ┆ Inspecc.de  ┆ Bloqueado │
│ ---      ┆ de material  ┆ ---    ┆ utilización ┆ ---     ┆ l.          ┆ calidad     ┆ ---       │
│ str      ┆ ---          ┆ str    ┆ ---         ┆ f64     ┆ ---         ┆ ---         ┆ f64       │
│          ┆ str          ┆        ┆ f64         ┆         ┆ i64         ┆ i64         ┆           │
╞══════════╪══════════════╪════════╪═════════════╪═════════╪═════════════╪═════════════╪═══════════╡
│ 122238   ┆ ROTULA SUP   ┆ 122238 ┆ 209.0       ┆ 0.0     ┆ 0           ┆ 0           ┆ 0.0       │
│          ┆ AMB DGP      ┆        ┆             ┆         ┆             ┆             ┆           │
│          ┆ TOYOTA HILU… ┆        ┆             ┆         ┆             ┆             ┆           │
└──────────┴──────────────┴────────┴─────────────┴─────────┴─────────────┴───

# Stock_Tiendas trabajado

1. **Agrupar por `Ult Eslabon`**:
   - Se suman las columnas `'Libre utilización'`, `'Trans./Trasl.'`, e `'Inspecc.de calidad'`.

2. **Crear columna `Stock Tiendas`**:
   - Se calcula la suma total de las tres columnas anteriores.

3. **Seleccionar columnas**:
   - Se seleccionan solo las columnas `Ult Eslabon` y `Stock Tiendas`.

4. **Renombrar columnas**:
   - Si alguna columna contiene `labon`, se renombra a `UE`.

5. **Resultado**:
   - El DataFrame final, `df_tiendas`, contiene solo `Ult Eslabon` y `Stock Tiendas`.

Este código agrupa, calcula y filtra las columnas necesarias para crear el DataFrame final.


In [103]:
df_tiendas_agrupado = df_tiendas.group_by('Ult Eslabon').agg(
    pl.col('Libre utilización').cast(pl.Float64).sum().alias('Suma Libre utilización'),
    pl.col('Trans./Trasl.').cast(pl.Float64).sum().alias('Suma Trans./Trasl.'),
    pl.col('Inspecc.de calidad').cast(pl.Float64).sum().alias('Suma Inspecc.de calidad')
)
stock_tiendas = df_tiendas_agrupado.with_columns(
    (pl.col('Suma Libre utilización') + pl.col('Suma Trans./Trasl.') + pl.col('Suma Inspecc.de calidad')).alias('Stock Tiendas')
)
df_tiendas = stock_tiendas.select(['Ult Eslabon', 'Stock Tiendas'])
df_tiendas = df_tiendas.rename({col: "UE" for col in df_tiendas.columns if "labon" in col.lower()})
print(df_tiendas[0])

shape: (1, 2)
┌────────┬───────────────┐
│ UE     ┆ Stock Tiendas │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ 117688 ┆ 8.0           │
└────────┴───────────────┘


## BUSCAR FALTANTES

In [104]:
import os
import polars as pl
from openpyxl import load_workbook

ruta = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Faltantes y Sobrantes Retail\Sobrantes y Faltantes AP - AGP")

archivos = [f for f in os.listdir(ruta) if f.lower().endswith('.xlsx')]

archivos.sort(reverse=True, key=lambda x: os.path.getmtime(os.path.join(ruta, x)))

archivo_reciente = os.path.join(ruta, archivos[0]) if archivos else None

if archivo_reciente:
    print(f"Leyendo el archivo: {archivo_reciente}")
    wb = load_workbook(archivo_reciente, read_only=True)
    hojas = wb.sheetnames
    
    hoja_tb_python = next((hoja for hoja in hojas if 'tb python' in hoja.lower()), None)

    if hoja_tb_python:
        faltantes = pl.read_excel(archivo_reciente, sheet_name=hoja_tb_python)
        print(f"Se leyó la hoja: {hoja_tb_python}")
    else:
        print("No se encontró una hoja que contenga 'tb python'.")
else:
    print("No se encontraron archivos Excel en la ruta.")


Leyendo el archivo: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Faltantes y Sobrantes Retail\Sobrantes y Faltantes AP - AGP\Falt & Sobr AP Y AGP_18.11.24.xlsx
Se leyó la hoja:  TB Python


In [105]:
faltantes = faltantes.rename({col: 'UE' for col in faltantes.columns if 'labon' in col})
print(faltantes[0])

shape: (1, 5)
┌─────┬─────────┬───────────┬──────────┬────────────────┐
│ POG ┆ UE      ┆ Faltante  ┆ Sobrante ┆ Stock Objetivo │
│ --- ┆ ---     ┆ ---       ┆ ---      ┆ ---            │
│ str ┆ str     ┆ i64       ┆ f64      ┆ i64            │
╞═════╪═════════╪═══════════╪══════════╪════════════════╡
│ AP  ┆ 1000143 ┆ 57        ┆ 0.0      ┆ 65             │
└─────┴─────────┴───────────┴──────────┴────────────────┘


## Limpieza transito

In [106]:
df_transito[0]

__UNNAMED__0,AUX,Status,Material,Texto breve,Cantidad,Centro,NomSector_actual,Origen,TIPO,Cl.documento compras,Fecha,Nombre de proveedor,Grupo de compras,Semana,Mes,Año
i64,str,str,str,str,i64,i64,str,str,str,str,datetime[ms],str,str,i64,i64,i64
0,"""470002295310""","""No Facturado""","""1067930""","""COOLANT AZUL -16 TRP CJ 6B 1GL…",0,714,"""Lubricantes""","""NAC""","""AFM""","""ZSPT""",2024-11-30 00:00:00,"""11460 COMERCIAL SURCHILE …","""TT3""",48,11,2024


## Si la semana del transito es 1 y el mes es 12 entonces sumar +1 al Año

In [107]:
df_transito = df_transito.with_columns(
    pl.when((pl.col("Semana") == 1) & (pl.col("Mes") == 12))
    .then(pl.col("Año") + 1)
    .otherwise(pl.col("Año"))
    .alias("Año")
)
print(df_transito[0])

shape: (1, 17)
┌──────────────┬──────────────┬──────────────┬──────────┬───┬──────────┬────────┬─────┬──────┐
│ __UNNAMED__0 ┆ AUX          ┆ Status       ┆ Material ┆ … ┆ Grupo de ┆ Semana ┆ Mes ┆ Año  │
│ ---          ┆ ---          ┆ ---          ┆ ---      ┆   ┆ compras  ┆ ---    ┆ --- ┆ ---  │
│ i64          ┆ str          ┆ str          ┆ str      ┆   ┆ ---      ┆ i64    ┆ i64 ┆ i64  │
│              ┆              ┆              ┆          ┆   ┆ str      ┆        ┆     ┆      │
╞══════════════╪══════════════╪══════════════╪══════════╪═══╪══════════╪════════╪═════╪══════╡
│ 0            ┆ 470002295310 ┆ No Facturado ┆ 1067930  ┆ … ┆ TT3      ┆ 48     ┆ 11  ┆ 2024 │
└──────────────┴──────────────┴──────────────┴──────────┴───┴──────────┴────────┴─────┴──────┘


In [108]:
df_transito = df_transito.group_by(['Material', 'Texto breve', 'Año','Semana']).agg(
    pl.col('Cantidad').sum().alias('Cantidad')
)
print(df_transito[0])

shape: (1, 5)
┌──────────┬─────────────────────────────────┬──────┬────────┬──────────┐
│ Material ┆ Texto breve                     ┆ Año  ┆ Semana ┆ Cantidad │
│ ---      ┆ ---                             ┆ ---  ┆ ---    ┆ ---      │
│ str      ┆ str                             ┆ i64  ┆ i64    ┆ i64      │
╞══════════╪═════════════════════════════════╪══════╪════════╪══════════╡
│ 123126   ┆ BAT HK MF40B19FL 35 AH 330 CCA… ┆ 2024 ┆ 49     ┆ 0        │
└──────────┴─────────────────────────────────┴──────┴────────┴──────────┘


In [109]:
df_transito = df_transito.with_columns(
    pl.col("Año").cast(pl.Utf8).str.slice(-2, 2).alias("Año")
).with_columns(
    (pl.lit("Sem ") + pl.col("Semana").cast(pl.Utf8) + pl.lit(" (") + pl.col("Año") + pl.lit(")")).alias("Sem")
)


## TUBO Y FALTANTES LISTO > VOY AHORA CON EL SP

### LEER SUPPLY PLAN

Este código busca el archivo Excel más reciente que contenga "SP" en su nombre dentro de una carpeta correspondiente al año y mes, según la semana actual:

1. **Semana 1 y 2**: Detecta el año y busca la carpeta del mes anterior.
2. **Semana 3 y 4**: Detecta el año y busca la carpeta del mes actual.
3. Dentro de la carpeta del mes, busca el archivo Excel más reciente con "SP" en el nombre y lo lee con Polars.


In [110]:
import os
import polars as pl
from datetime import datetime
from openpyxl import load_workbook

ruta = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Open to Buy_OTB")

meses_esp = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo", 6: "junio",
    7: "julio", 8: "agosto", 9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

mes_actual = datetime.now().month
anio_actual = datetime.now().year
mes_siguiente = mes_actual + 1 if mes_actual < 12 else 1
anio_mes = anio_actual if semana in [1, 2] else anio_actual if mes_siguiente > 1 else anio_actual + 1
nombre_mes = meses_esp[mes_actual] if semana in [1, 2] else meses_esp[mes_siguiente]

carpetas = [carpeta for carpeta in os.listdir(ruta) if os.path.isdir(os.path.join(ruta, carpeta)) and len(carpeta) == 4]

carpeta_entrada = next(
    (os.path.join(ruta, carpeta) for carpeta in carpetas if str(anio_mes) in carpeta), None
)

if carpeta_entrada:
    print(f"Ingresando a la carpeta: {carpeta_entrada}")
    carpetas_mes = [carpeta for carpeta in os.listdir(carpeta_entrada) if nombre_mes in carpeta.lower()]
    
    if carpetas_mes:
        carpeta_mes = os.path.join(carpeta_entrada, carpetas_mes[0])
        archivos = [f for f in os.listdir(carpeta_mes) if f.lower().endswith('.xlsx') and 'sp' in f.lower()]
        
        if archivos:
            archivo_reciente = max(archivos, key=lambda x: os.path.getmtime(os.path.join(carpeta_mes, x)))
            archivo_path = os.path.join(carpeta_mes, archivo_reciente)
            
            # Usar openpyxl para obtener los nombres de las hojas
            wb = load_workbook(archivo_path, read_only=True)
            hojas = wb.sheetnames

            # Buscar la hoja que contenga 'base' en el nombre
            hoja_base = next((hoja for hoja in hojas if 'base' in hoja.lower()), None)
            
            if hoja_base:
                df_supply_plan = pl.read_excel(archivo_path, sheet_name=hoja_base)
                print(f"Se leyó el archivo: {archivo_path}, hoja: {hoja_base}")
            else:
                print("No se encontró una hoja llamada 'base'.")
        else:
            print("No se encontró archivo con 'SP' en el nombre.")
    else:
        print(f"No se encontró la carpeta del mes: {nombre_mes}")
else:
    print(f"No se encontró carpeta correspondiente al año {anio_mes}")


Ingresando a la carpeta: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Open to Buy_OTB\2024
Se leyó el archivo: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Open to Buy_OTB\2024\12. Diciembre 2024\SP Diciembre 2024.xlsx, hoja: Base


In [111]:
if 'df_supply_plan' in locals() and df_supply_plan is not None:
    for i, row in enumerate(df_supply_plan.iter_rows(named=False)):
        if row[0] == "Material":
            nuevos_nombres = [str(col) for col in row]
            seen = {}
            for idx, col in enumerate(nuevos_nombres):
                if col in seen:
                    seen[col] += 1
                    nuevos_nombres[idx] = f"{col}_{seen[col]}"
                else:
                    seen[col] = 0
            df_supply_plan = df_supply_plan.slice(i + 1)
            df_supply_plan = df_supply_plan.rename({df_supply_plan.columns[j]: nuevos_nombres[j] for j in range(len(nuevos_nombres))})
            break


# EMPEZAR A CREAR BASE DE DISPO Y TAMBIEN DF DE PONDERACION

# Ponderación con FC PROM y %SKU FC

1. **Join:** Combina `df1` con `df_supply_plan` para agregar columna `ORIGEN`.
2. **Selección:** Extrae columnas clave y columnas adicionales (170 a 176) de `df1_con_origen`.
3. **Conversión:** Convierte las columnas seleccionadas a tipo `float` y reemplaza valores nulos con `0`.
4. **Cálculo de FC PROM:**
   - Si `ORIGEN` es "NAC", calcula promedio de las primeras 3 columnas adicionales.
   - De lo contrario, calcula promedio de las primeras 6 columnas adicionales.
5. **Cálculo de %SKU FC:**
   - Calcula el porcentaje de `FC PROM` sobre la suma de `FC PROM` dentro de cada `Ult. Eslabón`.
   - Rellena con `0` si el denominador es `0`.
6. **Resultado:** Genera el DataFrame `ponderacion` con columnas `FC PROM` y `%SKU FC`.


In [112]:
df1_con_origen = df1.join(
    df_supply_plan.select(["Material", "ORIGEN"]),
    left_on="Ult. Eslabón",
    right_on="Material",
    how="left"
)

ponderacion = df1_con_origen.select([
    "ID", 
    "Ult. Eslabón",
    "Vig", 
    "Canal",
    "VProm",
    "Stock Pond",
    "ORIGEN"
] + df1_con_origen.columns[170:176])

ponderacion = ponderacion.with_columns(
    [pl.col(df1_con_origen.columns[i]).cast(pl.Float64).fill_null(0) for i in range(170, 176)]
)

col_inicio = ponderacion.columns.index("ORIGEN") + 1
columnas_posteriores = ponderacion.columns[col_inicio:]

ponderacion = ponderacion.with_columns(
    pl.when(pl.col("ORIGEN") == "NAC")
    .then(
        sum(pl.col(col) for col in columnas_posteriores[:3]) / len(columnas_posteriores[:3])
    )
    .otherwise(
        sum(pl.col(col) for col in columnas_posteriores[:6]) / len(columnas_posteriores[:6])
    )
    .alias("FC PROM")
)

ponderacion = ponderacion.with_columns(
    (pl.col("FC PROM") / 
     pl.col("FC PROM").sum().over("Ult. Eslabón"))
    .fill_nan(0) 
    .alias("%SKU FC")
)
print(ponderacion.head(1))


shape: (1, 15)
┌──────────────┬─────────┬─────┬───────────┬───┬──────────────┬─────────────┬────────────┬─────────┐
│ ID           ┆ Ult.    ┆ Vig ┆ Canal     ┆ … ┆ 2025-04-01   ┆ 2025-05-01  ┆ FC PROM    ┆ %SKU FC │
│ ---          ┆ Eslabón ┆ --- ┆ ---       ┆   ┆ 00:00:00_6   ┆ 00:00:00_6  ┆ ---        ┆ ---     │
│ str          ┆ ---     ┆ str ┆ str       ┆   ┆ ---          ┆ ---         ┆ f64        ┆ f64     │
│              ┆ str     ┆     ┆           ┆   ┆ f64          ┆ f64         ┆            ┆         │
╞══════════════╪═════════╪═════╪═══════════╪═══╪══════════════╪═════════════╪════════════╪═════════╡
│ 1171331CL    ┆ 1171331 ┆ 1   ┆ CL RETAIL ┆ … ┆ 810.0        ┆ 810.0       ┆ 911.333333 ┆ 1.0     │
│ RETAIL       ┆         ┆     ┆           ┆   ┆              ┆             ┆            ┆         │
└──────────────┴─────────┴─────┴───────────┴───┴──────────────┴─────────────┴────────────┴─────────┘


In [113]:
df1.columns[170:176]

['2024-12-01 00:00:00_6',
 '2025-01-01 00:00:00_6',
 '2025-02-01 00:00:00_6',
 '2025-03-01 00:00:00_6',
 '2025-04-01 00:00:00_6',
 '2025-05-01 00:00:00_6']

OPCIONAL GUARDAR PONDERACION

In [114]:
# # Ruta de guardado
# ruta_guardado_csv = r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Downloads\ponderacion_modificado.csv"

# # Guardar como CSV con delimitador personalizado
# ponderacion.write_csv(
#     ruta_guardado_csv,
#     separator=';',  # Delimitador por punto y coma
#     float_precision=2,  # Controlar precisión decimal si es necesario
# )

# # Ajustar el separador decimal a ',' en los valores float
# # Convertimos el archivo para reemplazar los puntos decimales con comas
# with open(ruta_guardado_csv, 'r') as file:
#     contenido = file.read().replace('.', ',')

# # Guardar el contenido ajustado nuevamente
# with open(ruta_guardado_csv, 'w') as file:
#     file.write(contenido)

# print(f"Archivo CSV guardado con delimitador ';' y decimales ',' en: {ruta_guardado_csv}")


## calcular FC Ponderado

In [115]:
ponderacion.columns

['ID',
 'Ult. Eslabón',
 'Vig',
 'Canal',
 'VProm',
 'Stock Pond',
 'ORIGEN',
 '2024-12-01 00:00:00_6',
 '2025-01-01 00:00:00_6',
 '2025-02-01 00:00:00_6',
 '2025-03-01 00:00:00_6',
 '2025-04-01 00:00:00_6',
 '2025-05-01 00:00:00_6',
 'FC PROM',
 '%SKU FC']

# Base_Dispo

1. **Selección:** Columnas clave de `df1`.
2. **Join:** Combina con `ponderacion` para agregar `%SKU FC`.
3. **Renombrar:** `%SKU FC` → `Ponderación por canal`.
4. **Reorganización:** Ajuste de orden de columnas.
5. **Transformación:** Crea `Canal Consolidado` con reglas específicas.
6. **Validación:** Imprime valores únicos de `Canal Consolidado`.


In [116]:

base_dispo = df1.select([
    "ID",
    "Ult. Eslabón",
    "Descripcion Material",
    "Marca",
    "Vig",
    "Canal"   
])
base_dispo = base_dispo.join(
    ponderacion.select(["Ult. Eslabón", "%SKU FC"]),
    left_on="Ult. Eslabón",
    right_on="Ult. Eslabón",
    how="inner"
)
base_dispo = base_dispo.rename({"%SKU FC": "Ponderación por canal"})
column_order = [
    "ID",
    "Ult. Eslabón",
    "Descripcion Material",
    "Marca",
    "Ponderación por canal",
    "Vig",
    "Canal"
]
base_dispo = base_dispo.select(column_order)
base_dispo = base_dispo.with_columns(
   pl.col("Canal").map_elements(lambda x:
       "RETAIL" if x == "CL RETAIL" else
       "MAY" if x in ["CL MAYORISTA", "CL CES 01"] else
       "GT" if x in ["CL EASY", "CL SODIMAC", "CL WALMART", "CL SMU", "CL TOTTUS"] else
       "AGROPLANET" if x == "CL AGROPLANET" else
       "0",
       return_dtype=pl.Utf8
   ).alias("Canal Consolidado")
)
print(base_dispo.select("Canal Consolidado").unique())


shape: (3, 1)
┌───────────────────┐
│ Canal Consolidado │
│ ---               │
│ str               │
╞═══════════════════╡
│ MAY               │
│ GT                │
│ RETAIL            │
└───────────────────┘


AYUDA PARA SABER QUE COLUMAS NO ESTA ENCONTRANDO (OPCIONAL)

In [117]:
# Obtener las columnas disponibles en el DataFrame
columnas_disponibles = df_supply_plan.columns

# Lista de columnas que quieres seleccionar
columnas_deseadas = [
    "Sector",
    "Agrupación ClaCom",
    "Categoría ClaCom",
    "SubCategoría ClaCom",
    "Bobinas/Implementos/Otros",
    "COD. PROVEEDOR",
    "PROVEEDOR",
    "ORIGEN",
    "LEAD TIME",
    "Codigos Inquebrables",
    "Comentario Sesión",
    "Comentario Finales",
    "OBS Retail",
    "OBS DERCO"
]

# Identificar columnas no encontradas
columnas_no_encontradas = [col for col in columnas_deseadas if col not in columnas_disponibles]

columnas_no_encontradas


[]

### Filtrado y Renombramiento Dinámico de Columnas en `df_supply_plan`

Este script ajusta un DataFrame específico (`df_supply_plan`) para identificar, renombrar y filtrar columnas basadas en palabras clave.

#### Objetivo:
1. **Identificar columnas relevantes** según contenido parcial en sus nombres:
   - `"segmentaci"` (sin incluir "abcd") → Renombrada como `"Segmentacion AFM"`.
   - `"6 meses"` → Renombrada como `"Prom Forecast 3-6-12"`.
   - `"4 - 6 - 12"` → Renombrada como `"Prom Venta 3-6-12"`.
   - `"Codigos Inquebrables"` → Renombrada como `"Codigos Transparentes"`.

2. **Filtrar columnas** seleccionadas:
   - Fijas: `"Sector"`, `"Agrupación ClaCom"`, `"Categoría ClaCom"`, `"SubCategoría ClaCom"`, `"Bobinas/Implementos/Otros"`, `"COD. PROVEEDOR"`, `"PROVEEDOR"`, `"ORIGEN"`, `"LEAD TIME"`, `"Comentario Sesión"`, `"Comentario Finales"`, `"OBS Retail"`, `"OBS DERCO"`.
   - Dinámicas: Agregadas automáticamente si fueron encontradas y renombradas.

#### Resultado:
Un DataFrame filtrado con columnas relevantes para análisis y reportes.


In [118]:
renombramientos = {}

columna_segmentacion = [col for col in df_supply_plan.columns if "segmentaci" in col.lower() and "abcd" not in col.lower()]
if columna_segmentacion:
    renombramientos[columna_segmentacion[0]] = "Segmentacion AFM"

columna_forecast = [col for col in df_supply_plan.columns if "6 meses" in col.lower()]
if columna_forecast:
    renombramientos[columna_forecast[0]] = "Prom Forecast 3-6-12"

columna_venta = [col for col in df_supply_plan.columns if "4 - 6 - 12" in col]
if columna_venta:
    renombramientos[columna_venta[0]] = "Prom Venta 3-6-12"

columna_codigos = [col for col in df_supply_plan.columns if "Codigos Inquebrables" in col]
if columna_codigos:
    renombramientos[columna_codigos[0]] = "Codigos Transparentes"

if renombramientos:
    df_supply_plan = df_supply_plan.rename(renombramientos)

columnas_seleccionadas = [
    "Material",
    "Sector",
    "Agrupación ClaCom",
    "SubAgrupación ClaCom",
    "Categoría ClaCom",
    "SubCategoría ClaCom",
    "Bobinas/Implementos/Otros",
    "COD. PROVEEDOR",
    "PROVEEDOR",
    "ORIGEN",
    "LEAD TIME",
    "Codigos Transparentes",
    "Comentario Sesión",
    "Comentario Finales",
    "OBS Retail",
    "OBS DERCO"
]

if "Segmentacion AFM" in renombramientos.values():
    columnas_seleccionadas.append("Segmentacion AFM")
if "Prom Forecast 3-6-12" in renombramientos.values():
    columnas_seleccionadas.append("Prom Forecast 3-6-12")
if "Prom Venta 3-6-12" in renombramientos.values():
    columnas_seleccionadas.append("Prom Venta 3-6-12")

df_supply_plan_filtrado = df_supply_plan.select(columnas_seleccionadas)


In [119]:
base_dispo = base_dispo.rename({"Ult. Eslabón": "Material"}).join(
    df_supply_plan_filtrado, 
    on="Material", 
    how="left"
).rename({"Material": "Ult. Eslabón"})


In [120]:
base_dispo.head(1)

ID,Ult. Eslabón,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25"""


# Expansión, Filtrado y Agrupación de Datos con Polars

Este código realiza las siguientes operaciones:
1. Genera todas las semanas correspondientes a los meses de una columna `variable` que contiene fechas.
2. Filtra filas según condiciones específicas: elimina semanas 1 en diciembre de los años 2024, 2025, 2026 y 2027.
3. Agrupa los datos por columnas clave y calcula el promedio de `Fc Semanal`.

In [121]:
import polars as pl
from datetime import datetime, timedelta
from calendar import monthrange

def semanas_del_mes(fecha):
    fecha_dt = datetime.strptime(fecha, "%Y-%m-%d")
    _, dias_mes = monthrange(fecha_dt.year, fecha_dt.month)
    inicio_mes = datetime(fecha_dt.year, fecha_dt.month, 1)
    fin_mes = datetime(fecha_dt.year, fecha_dt.month, dias_mes)

    semanas = []
    fecha_actual = inicio_mes
    while fecha_actual <= fin_mes:
        semana_iso = fecha_actual.isocalendar().week
        if semana_iso not in semanas:
            semanas.append(semana_iso)
        fecha_actual += timedelta(days=1)

    return semanas

def expandir_df(df):
    df_pd = df.to_pandas()
    df_expanded = df_pd.assign(Semanas=df_pd["variable"].dt.strftime('%Y-%m-%d').apply(semanas_del_mes)).explode("Semanas")
    return pl.from_pandas(df_expanded)

df_melt_1_expanded = expandir_df(df_melt_1)
fc_sem = df_melt_1_expanded
fc_sem = fc_sem.with_columns(
    pl.col("variable").dt.year().alias("Año")
)
fc_sem = fc_sem.filter(~(
    ((pl.col("Año") == 2024) | (pl.col("Año") == 2025) | (pl.col("Año") == 2026) | (pl.col("Año") == 2027)) &
    (pl.col("Semanas") == 1) &
    (pl.col("variable").dt.month() == 12)
))
fc_sem = fc_sem.with_columns(
    pl.col("Fc Semanal").fill_null(0)
)
fc_sem_agrupado = fc_sem.group_by(
    ["ID", "Ult. Eslabón", "Año", "Semanas"]
).agg(
    pl.col("Fc Semanal").mean().alias("Promedio Fc Semanal")
)

print(fc_sem_agrupado[0])


shape: (1, 5)
┌──────────────────┬──────────────┬──────┬─────────┬─────────────────────┐
│ ID               ┆ Ult. Eslabón ┆ Año  ┆ Semanas ┆ Promedio Fc Semanal │
│ ---              ┆ ---          ┆ ---  ┆ ---     ┆ ---                 │
│ str              ┆ str          ┆ i32  ┆ i64     ┆ f64                 │
╞══════════════════╪══════════════╪══════╪═════════╪═════════════════════╡
│ 1069054CL RETAIL ┆ 1069054      ┆ 2025 ┆ 23      ┆ 0.0                 │
└──────────────────┴──────────────┴──────┴─────────┴─────────────────────┘


# Validar valores unicos del fc_sem

In [122]:
# fc_sem_agrupado['variable'].unique()

## Vrificar valores unicos de semanas según fecha

In [123]:
# valores_unicos = fc_sem.filter(
#     # (pl.col("variable") == pl.datetime(2024, 12, 1)) |
#     (pl.col("variable") == pl.datetime(2024, 12, 1)) 
#     # (pl.col("variable") == pl.datetime(2025, 1, 1))
# ).select("Semanas").unique()

# print(valores_unicos)


## VALIDACION FC_SEM Y AGRUPADO OPCIONAL A CSV GUARDAR

In [124]:
# fc_sem.to_pandas().to_csv(
#     r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Downloads\fc_sem.csv",
#     sep=';',
#     decimal=',',
#     index=False
# )

# fc_sem_agrupado.to_pandas().to_csv(
#     r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Downloads\fc_sem_agrupado.csv",
#     sep=';',
#     decimal=',',
#     index=False
# )


In [125]:
fc_sem_agrupado.head(1)

ID,Ult. Eslabón,Año,Semanas,Promedio Fc Semanal
str,str,i32,i64,f64
"""1069054CL RETAIL""","""1069054""",2025,23,0.0


In [126]:
print(
    fc_sem_agrupado.filter(
        (pl.col("ID") == "1000147CL RETAIL") &
        (pl.col("Semanas") == 48) &
        (pl.col("Año") == 2024)
    )
)

shape: (1, 5)
┌──────────────────┬──────────────┬──────┬─────────┬─────────────────────┐
│ ID               ┆ Ult. Eslabón ┆ Año  ┆ Semanas ┆ Promedio Fc Semanal │
│ ---              ┆ ---          ┆ ---  ┆ ---     ┆ ---                 │
│ str              ┆ str          ┆ i32  ┆ i64     ┆ f64                 │
╞══════════════════╪══════════════╪══════╪═════════╪═════════════════════╡
│ 1000147CL RETAIL ┆ 1000147      ┆ 2024 ┆ 48      ┆ 12.01276            │
└──────────────────┴──────────────┴──────┴─────────┴─────────────────────┘


In [127]:
fc_sem_agrupado = fc_sem_agrupado.with_columns(
    pl.col("Año").cast(pl.Utf8).str.slice(-2, 2).alias("Año")
).with_columns(
    (pl.lit("Sem ") + pl.col("Semanas").cast(pl.Utf8) + pl.lit(" (") + pl.col("Año") + pl.lit(")")).alias("Sem")
)
print(fc_sem_agrupado[0])


shape: (1, 6)
┌──────────────────┬──────────────┬─────┬─────────┬─────────────────────┬─────────────┐
│ ID               ┆ Ult. Eslabón ┆ Año ┆ Semanas ┆ Promedio Fc Semanal ┆ Sem         │
│ ---              ┆ ---          ┆ --- ┆ ---     ┆ ---                 ┆ ---         │
│ str              ┆ str          ┆ str ┆ i64     ┆ f64                 ┆ str         │
╞══════════════════╪══════════════╪═════╪═════════╪═════════════════════╪═════════════╡
│ 1069054CL RETAIL ┆ 1069054      ┆ 25  ┆ 23      ┆ 0.0                 ┆ Sem 23 (25) │
└──────────────────┴──────────────┴─────┴─────────┴─────────────────────┴─────────────┘


In [128]:
fc=fc_sem_agrupado
# Cambiar el nombre de las columnas en 'fc' que contengan 'labon' en su nombre
fc = fc.rename({col: "UE" for col in fc.columns if "labón" in col.lower()})

# Cambiar el nombre de las columnas en 'df_transito' que contengan 'material' en su nombre
df_transito = df_transito.rename({col: "UE" for col in df_transito.columns if "Material" in col.lower()})

# Cambiar el nombre de las columnas en 'base_dispo' que contengan 'labón' en su nombre
base_dispo = base_dispo.rename({col: "UE" for col in base_dispo.columns if "labón" in col.lower()})



## PRINT TUBO Y FALTANTES Y FC

In [129]:
print(df_711[0])
print(df_stock[0])
print(faltantes[0])

shape: (1, 7)
┌──────────┬───────────────────────┬────────┬─────────────┬───────────────┬────────────┬───────────┐
│ Material ┆ Texto breve de        ┆ UE     ┆ Libre       ┆ Trans./Trasl. ┆ Inspecc.de ┆ Bloqueado │
│ ---      ┆ material              ┆ ---    ┆ utilización ┆ ---           ┆ calidad    ┆ ---       │
│ str      ┆ ---                   ┆ str    ┆ ---         ┆ i64           ┆ ---        ┆ f64       │
│          ┆ str                   ┆        ┆ f64         ┆               ┆ i64        ┆           │
╞══════════╪═══════════════════════╪════════╪═════════════╪═══════════════╪════════════╪═══════════╡
│ 130730   ┆ FILTRO AIRE INT. NH   ┆ 130730 ┆ 6.0         ┆ 0             ┆ 0          ┆ 0.0       │
└──────────┴───────────────────────┴────────┴─────────────┴───────────────┴────────────┴───────────┘
shape: (1, 8)
┌──────────┬──────────────┬────────┬─────────────┬─────────┬─────────────┬─────────────┬───────────┐
│ Material ┆ Texto breve  ┆ UE     ┆ Libre       ┆ Control ┆ Tr

In [130]:
print(df_transito[0])
print(fc[0])
print(df_tiendas[0])

shape: (1, 6)
┌──────────┬─────────────────────────────────┬─────┬────────┬──────────┬─────────────┐
│ Material ┆ Texto breve                     ┆ Año ┆ Semana ┆ Cantidad ┆ Sem         │
│ ---      ┆ ---                             ┆ --- ┆ ---    ┆ ---      ┆ ---         │
│ str      ┆ str                             ┆ str ┆ i64    ┆ i64      ┆ str         │
╞══════════╪═════════════════════════════════╪═════╪════════╪══════════╪═════════════╡
│ 123126   ┆ BAT HK MF40B19FL 35 AH 330 CCA… ┆ 24  ┆ 49     ┆ 0        ┆ Sem 49 (24) │
└──────────┴─────────────────────────────────┴─────┴────────┴──────────┴─────────────┘
shape: (1, 6)
┌──────────────────┬─────────┬─────┬─────────┬─────────────────────┬─────────────┐
│ ID               ┆ UE      ┆ Año ┆ Semanas ┆ Promedio Fc Semanal ┆ Sem         │
│ ---              ┆ ---     ┆ --- ┆ ---     ┆ ---                 ┆ ---         │
│ str              ┆ str     ┆ str ┆ i64     ┆ f64                 ┆ str         │
╞══════════════════╪═════════╪═

# Explicación de cruces
1. **Unión con `df_711`**:
   - Se agrega la columna `Libre utilización` de `df_711` a `base_dispo`, renombrándola como `711` y reemplazando valores nulos por 0.

2. **Unión con `df_stock`**:
   - Se agregan las columnas `Libre utilización` (renombrada como `Stock CD`) y `Control` de `df_stock`.

3. **Unión con `df_tiendas`**:
   - Se agrega la columna `Stock Tiendas` de `df_tiendas`.

4. **Limpieza de columnas en `faltantes`**:
   - Se eliminan espacios al final de los nombres de las columnas en `faltantes`.

5. **Unión con `faltantes`**:
   - Se agregan las columnas `Faltante`, `Sobrante`, y `Stock Objetivo` de `faltantes`.

6. **Reemplazo de `null` por 0**:
   - Se reemplazan los valores `null` por `0` en las columnas especificadas: `711`, `Stock CD`, `Control`, `Stock Tiendas`, `Faltante`, `Sobrante`, y `Stock Objetivo`.

Este código une varias tablas y limpia los valores nulos en las columnas clave de `base_dispo`.


In [ ]:
base_dispo = base_dispo.join(
    df_711.select(["UE", "Libre utilización"]),
    on="UE",
    how="left"
)

base_dispo = base_dispo.rename({"Libre utilización": "711"})
base_dispo = base_dispo.with_columns(
    pl.col("711").fill_null(0)
)
base_dispo = base_dispo.join(
    df_stock.select(["UE", "Libre utilización", "Control"]),
    on="UE",
    how="left"
)

base_dispo = base_dispo.rename({
    "Libre utilización": "Stock CD",
    "Control": "Control"
})
base_dispo = base_dispo.with_columns(
    pl.when(pl.col("Canal") == "CL RETAIL")
    .then(
        base_dispo.join(
            df_tiendas.select(["UE", "Stock Tiendas"]),
            on="UE",
            how="left"
        )["Stock Tiendas"]
    )
    .otherwise(pl.lit(None))
    .alias("Stock Tiendas")
)


faltantes.columns = [col.strip() for col in faltantes.columns]
base_dispo = base_dispo.join(
    faltantes.select(["UE", "Faltante", "Sobrante", "Stock Objetivo"]),
    on="UE",
    how="left"
)
base_dispo = base_dispo.with_columns([
    pl.col(col).fill_null(0) for col in ["711", "Stock CD", "Control", "Stock Tiendas", "Faltante", "Sobrante", "Stock Objetivo"]
])
print(base_dispo[0])


shape: (1, 33)
┌───────────┬─────────┬─────────────────────┬───────┬───┬─────────┬──────────┬──────────┬──────────┐
│ ID        ┆ UE      ┆ Descripcion         ┆ Marca ┆ … ┆ Stock   ┆ Faltante ┆ Sobrante ┆ Stock    │
│ ---       ┆ ---     ┆ Material            ┆ ---   ┆   ┆ Tiendas ┆ ---      ┆ ---      ┆ Objetivo │
│ str       ┆ str     ┆ ---                 ┆ str   ┆   ┆ ---     ┆ i64      ┆ f64      ┆ ---      │
│           ┆         ┆ str                 ┆       ┆   ┆ f64     ┆          ┆          ┆ i64      │
╞═══════════╪═════════╪═════════════════════╪═══════╪═══╪═════════╪══════════╪══════════╪══════════╡
│ 1171331CL ┆ 1171331 ┆ SCENT BOMB 2 PACK   ┆ SCENT ┆ … ┆ 1446.0  ┆ 48       ┆ 1133.0   ┆ 292      │
│ RETAIL    ┆         ┆ PAPER MAN TR…       ┆ BOMB  ┆   ┆         ┆          ┆          ┆          │
└───────────┴─────────┴─────────────────────┴───────┴───┴─────────┴──────────┴──────────┴──────────┘


## ORDENAR TRANSITO

In [132]:
min_semana_2024 = df_transito.filter(pl.col("Año") == "24").select(pl.col("Semana").min()).to_numpy()

print(f"La menor semana del año 2024 es: {min_semana_2024[0][0]}")


La menor semana del año 2024 es: 47


In [133]:
df_transito = df_transito.sort(by="Semana", descending=False)

print(df_transito)


shape: (12_121, 6)
┌──────────┬─────────────────────────────────┬─────┬────────┬──────────┬─────────────┐
│ Material ┆ Texto breve                     ┆ Año ┆ Semana ┆ Cantidad ┆ Sem         │
│ ---      ┆ ---                             ┆ --- ┆ ---    ┆ ---      ┆ ---         │
│ str      ┆ str                             ┆ str ┆ i64    ┆ i64      ┆ str         │
╞══════════╪═════════════════════════════════╪═════╪════════╪══════════╪═════════════╡
│ 457595   ┆ BAT AUTOSTYLE   MF105D31R 780C… ┆ 25  ┆ 1      ┆ 588      ┆ Sem 1 (25)  │
│ 120412   ┆ NEUM LTR FTN 245/75R16 111T FS… ┆ 25  ┆ 1      ┆ 0        ┆ Sem 1 (25)  │
│ 130079   ┆ HORQUILLA EMBRAGUE              ┆ 25  ┆ 1      ┆ 0        ┆ Sem 1 (25)  │
│ 1054081  ┆ ESPONJA DE MALLA MOMO           ┆ 25  ┆ 1      ┆ 300      ┆ Sem 1 (25)  │
│ 121862   ┆ JGO PASTILLA FRENO DGP PRO DXP… ┆ 25  ┆ 1      ┆ 0        ┆ Sem 1 (25)  │
│ …        ┆ …                               ┆ …   ┆ …      ┆ …        ┆ …           │
│ 1163846  ┆ AROM SUPER 

In [134]:
df_transito = df_transito.rename({"Material": "UE"})
df_transito[0]

UE,Texto breve,Año,Semana,Cantidad,Sem
str,str,str,i64,i64,str
"""457595""","""BAT AUTOSTYLE MF105D31R 780C…","""25""",1,588,"""Sem 1 (25)"""


## FC PIVOT

In [135]:
fc.columns

['ID', 'UE', 'Año', 'Semanas', 'Promedio Fc Semanal', 'Sem']

In [136]:
fc_pvt = fc.pivot(
    values="Promedio Fc Semanal",
    index=["ID"],
    on="Sem"
)


In [137]:
fc_pvt = fc_pvt.fill_null(0)
fc_pvt = fc_pvt.rename({col: f"{col}_fc" for col in fc_pvt.columns if col.startswith("Sem")})
fc_pvt[0]

ID,Sem 23 (25)_fc,Sem 52 (24)_fc,Sem 24 (25)_fc,Sem 47 (25)_fc,Sem 39 (25)_fc,Sem 17 (25)_fc,Sem 49 (24)_fc,Sem 37 (25)_fc,Sem 31 (25)_fc,Sem 26 (25)_fc,Sem 44 (25)_fc,Sem 46 (24)_fc,Sem 48 (24)_fc,Sem 28 (25)_fc,Sem 30 (25)_fc,Sem 4 (25)_fc,Sem 51 (24)_fc,Sem 42 (25)_fc,Sem 14 (25)_fc,Sem 46 (25)_fc,Sem 36 (25)_fc,Sem 6 (25)_fc,Sem 21 (25)_fc,Sem 20 (25)_fc,Sem 27 (25)_fc,Sem 33 (25)_fc,Sem 38 (25)_fc,Sem 32 (25)_fc,Sem 45 (24)_fc,Sem 18 (25)_fc,Sem 9 (25)_fc,Sem 11 (25)_fc,Sem 19 (25)_fc,Sem 5 (25)_fc,Sem 13 (25)_fc,Sem 43 (25)_fc,Sem 22 (25)_fc,Sem 44 (24)_fc,Sem 25 (25)_fc,Sem 50 (24)_fc,Sem 34 (25)_fc,Sem 40 (25)_fc,Sem 1 (25)_fc,Sem 7 (25)_fc,Sem 47 (24)_fc,Sem 12 (25)_fc,Sem 35 (25)_fc,Sem 41 (25)_fc,Sem 10 (25)_fc,Sem 48 (25)_fc,Sem 2 (25)_fc,Sem 15 (25)_fc,Sem 45 (25)_fc,Sem 3 (25)_fc,Sem 16 (25)_fc,Sem 29 (25)_fc,Sem 8 (25)_fc
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1069054CL RETAIL""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TRANSITO PIVOT

In [138]:
import polars as pl

# Agrupar el DataFrame por 'UE' y 'Sem', y calcular la suma de 'Cantidad'
df_transito_agrupado = (
    df_transito.group_by(['UE', 'Sem'])
    .agg(pl.col('Cantidad').sum().alias('Cantidad'))
)

# Mostrar el resultado
print(df_transito_agrupado)
df_transito=df_transito_agrupado


shape: (11_988, 3)
┌─────────┬─────────────┬──────────┐
│ UE      ┆ Sem         ┆ Cantidad │
│ ---     ┆ ---         ┆ ---      │
│ str     ┆ str         ┆ i64      │
╞═════════╪═════════════╪══════════╡
│ 527091  ┆ Sem 51 (24) ┆ 2        │
│ 547524  ┆ Sem 51 (24) ┆ 1        │
│ 141345  ┆ Sem 14 (25) ┆ 24       │
│ 761652  ┆ Sem 51 (24) ┆ 1        │
│ 1126335 ┆ Sem 4 (25)  ┆ 2400     │
│ …       ┆ …           ┆ …        │
│ 120933  ┆ Sem 8 (25)  ┆ 5100     │
│ 121017  ┆ Sem 50 (24) ┆ 500      │
│ 530946  ┆ Sem 51 (24) ┆ 1        │
│ 143195  ┆ Sem 11 (25) ┆ 189      │
│ 502835  ┆ Sem 51 (24) ┆ 1        │
└─────────┴─────────────┴──────────┘


In [139]:
df_transito=df_transito_agrupado

In [140]:
df_transito_pivot = df_transito.pivot(
    values="Cantidad",
    index=["UE"],
    on="Sem"
)
transito_pvt = df_transito_pivot

In [141]:
transito_pvt = transito_pvt.fill_null(0)
transito_pvt = transito_pvt.rename({col: f"{col}_tr" for col in transito_pvt.columns if col.startswith("Sem")})
transito_pvt[0]

UE,Sem 51 (24)_tr,Sem 14 (25)_tr,Sem 4 (25)_tr,Sem 5 (25)_tr,Sem 7 (25)_tr,Sem 2 (25)_tr,Sem 16 (25)_tr,Sem 12 (25)_tr,Sem 13 (25)_tr,Sem 15 (25)_tr,Sem 21 (25)_tr,Sem 8 (25)_tr,Sem 1 (25)_tr,Sem 49 (24)_tr,Sem 6 (25)_tr,Sem 52 (24)_tr,Sem 3 (25)_tr,Sem 11 (25)_tr,Sem 10 (25)_tr,Sem 9 (25)_tr,Sem 48 (24)_tr,Sem 17 (25)_tr,Sem 50 (24)_tr,Sem 47 (24)_tr,Sem 19 (25)_tr,Sem 18 (25)_tr,Sem 22 (25)_tr,Sem 20 (25)_tr
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""527091""",2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# CREAR SEMANA 1 

In [142]:
import datetime

fecha_actual = datetime.date.today()
semana_actual = fecha_actual.isocalendar()[1]

print(f"La semana actual es: {semana_actual}")

fecha_actual = datetime.date.today()
semana_actual = fecha_actual.isocalendar()[1]
año_actual = str(fecha_actual.year)[-2:]

columna_nombre = f"Sem {semana_actual} ({año_actual})"

base_dispo = base_dispo.with_columns(
    pl.when(pl.col("Canal Consolidado") == "RETAIL")
    .then(
        pl.col("Stock CD") * pl.col("Ponderación por canal") - 
        (pl.col("Faltante") if faltan == 1 else pl.lit(0)) +
        (pl.col("Stock Tiendas") if tienda == 1 else pl.lit(0))
    )
    .otherwise(pl.col("Stock CD") * pl.col("Ponderación por canal"))
    .alias(columna_nombre)
)

print(base_dispo)


La semana actual es: 47
shape: (60_746, 34)
┌─────────────┬─────────┬─────────────┬────────────┬───┬──────────┬──────────┬────────────┬────────┐
│ ID          ┆ UE      ┆ Descripcion ┆ Marca      ┆ … ┆ Faltante ┆ Sobrante ┆ Stock      ┆ Sem 47 │
│ ---         ┆ ---     ┆ Material    ┆ ---        ┆   ┆ ---      ┆ ---      ┆ Objetivo   ┆ (24)   │
│ str         ┆ str     ┆ ---         ┆ str        ┆   ┆ i64      ┆ f64      ┆ ---        ┆ ---    │
│             ┆         ┆ str         ┆            ┆   ┆          ┆          ┆ i64        ┆ f64    │
╞═════════════╪═════════╪═════════════╪════════════╪═══╪══════════╪══════════╪════════════╪════════╡
│ 1171331CL   ┆ 1171331 ┆ SCENT BOMB  ┆ SCENT BOMB ┆ … ┆ 48       ┆ 1133.0   ┆ 292        ┆ -48.0  │
│ RETAIL      ┆         ┆ 2 PACK      ┆            ┆   ┆          ┆          ┆            ┆        │
│             ┆         ┆ PAPER MAN   ┆            ┆   ┆          ┆          ┆            ┆        │
│             ┆         ┆ TR…         ┆        

# CREAR SEMANA 2


Este código realiza varias operaciones sobre un DataFrame llamado `base_dispo`, integrando datos de otros DataFrames (`transito_pvt` y `fc_pvt`), y ajustando las columnas de acuerdo a ciertos cálculos. A continuación se describe el flujo de operaciones:

1. **Creación de nuevas columnas**: 
   Se definen los nombres de las nuevas columnas `columna_tr` y `columna_fc`, que son variantes de la columna `columna_nombre_siguiente`.

2. **Unión de datos desde `transito_pvt`**:
   Si la columna `columna_tr` existe en el DataFrame `transito_pvt`, se realiza una unión (left join) con `base_dispo` utilizando la columna `"UE"` como clave de unión. Si no se encuentra la columna, se imprime un mensaje de advertencia.

3. **Unión de datos desde `fc_pvt`**:
   Similar al paso anterior, si la columna `columna_fc` existe en `fc_pvt`, se realiza otra unión con `base_dispo`, pero en este caso usando la columna `"ID"` como clave de unión. Si la columna no se encuentra, se imprime otro mensaje de advertencia.

4. **Condición para valores negativos**:
   Se crea una nueva columna `columna_nombre_siguiente`, en la que los valores de `columna_nombre` mayores o iguales a 0 se mantienen, mientras que los valores negativos se reemplazan por `None`.

5. **Suma con ponderaciones**:
   Se actualiza `columna_nombre_siguiente` sumando:
   - El valor actual de `columna_nombre_siguiente`.
   - La multiplicación de `columna_tr` por la columna `"Ponderación por canal"`.
   - Los valores de `columna_fc`, con los nulos reemplazados por 0.
   - La multiplicación de la columna `"Control"` por la columna `"Ponderación por canal"`.
   
6. **Eliminación de columnas temporales**:
   Se eliminan las columnas `columna_tr` y `columna_fc` del DataFrame después de realizar los cálculos.

Este flujo de trabajo asegura que los datos de `transito_pvt` y `fc_pvt` sean correctamente integrados y que los cálculos sean realizados adecuadamente con las ponderaciones aplicadas.

In [143]:
dix=base_dispo
dix

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24)
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-2400""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGM

In [144]:
# Obtener el nombre de la nueva columna: 'Sem (semana actual + 1) (yy)'
columna_nombre_siguiente = f"Sem {semana_actual + 1} ({año_actual})"
columna_nombre_siguiente
columna_tr = columna_nombre_siguiente + "_tr"
columna_fc = columna_nombre_siguiente + "_fc"

if columna_tr in transito_pvt.columns:
    base_dispo = base_dispo.join(
        transito_pvt.filter(
            pl.col("UE").is_in(base_dispo["UE"])
        ).select([pl.col("UE"), pl.col(columna_tr).alias(columna_tr)]),
        on="UE",
        how="left"
    )
else:
    print(f"Columna {columna_tr} no encontrada en transito_pvt.")

In [145]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24)_tr
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,i64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,0
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,0
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,0
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,0
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,null
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,null
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-2400""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Nece

In [146]:


if columna_fc in fc_pvt.columns:
    base_dispo = base_dispo.join(
        fc_pvt.filter(
            pl.col("ID").is_in(base_dispo["ID"])
        ).select([pl.col("ID"), pl.col(columna_fc).alias(columna_fc)]),
        on="ID",
        how="left"
    )
else:
    print(f"Columna {columna_fc} no encontrada en fc_pvt.")

In [147]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24)_tr,Sem 48 (24)_fc
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,i64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,0,187.066975
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,0,187.066975
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,0,187.066975
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,0,69.284065
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,0,92.378753
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,null,0.0
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,null,0.0
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-2400""","""Implementos""","

In [148]:



base_dispo = base_dispo.with_columns(
    pl.col(columna_nombre).alias(columna_nombre_siguiente)
)


base_dispo = base_dispo.with_columns(
    (
        pl.when(
            (pl.col(columna_nombre_siguiente) +
             (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1)) +
             (pl.col("Control").fill_null(0) * pl.col("Ponderación por canal").fill_null(1))
            ) > 0
        )
        .then(
            pl.col(columna_nombre_siguiente) +
            (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1)) +
            (pl.col("Control").fill_null(0) * pl.col("Ponderación por canal").fill_null(1)) -
            pl.col(columna_fc).fill_null(0)
        )
        .otherwise(
            pl.col(columna_nombre_siguiente) +
            (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1)) +
            (pl.col("Control").fill_null(0) * pl.col("Ponderación por canal").fill_null(1))
        )
        .alias(columna_nombre_siguiente)
    )
)


base_dispo = base_dispo.drop([columna_tr, columna_fc])


In [149]:
base_dispo[0]

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24)
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0


# CREAR SEMANA 3

In [150]:
sem_tres = f"Sem {semana_actual + 2} ({año_actual})"
sem_tres

'Sem 49 (24)'

In [151]:
sem_dos = f"Sem {semana_actual + 1} ({año_actual})"
sem_dos

'Sem 48 (24)'

In [152]:
columna_tr = sem_tres + "_tr"
columna_fc = sem_tres + "_fc"

## Traer transito sem3

In [153]:
if columna_tr in transito_pvt.columns:
    base_dispo = base_dispo.join(
        transito_pvt.filter(
            pl.col("UE").is_in(base_dispo["UE"])
        ).select([pl.col("UE"), pl.col(columna_tr).alias(columna_tr)]),
        on="UE",
        how="left"
    )
else:
    print(f"Columna {columna_tr} no encontrada en transito_pvt.")



In [154]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24),Sem 49 (24)_tr
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64,i64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0,2592
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,-144.0,2592
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,-24.0,2592
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,134.715935,768
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,99.621247,768
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-2400""","""Implementos""","""7

## Traer FC Sem 3

In [155]:
if columna_fc in fc_pvt.columns:
    base_dispo = base_dispo.join(
        fc_pvt.filter(
            pl.col("ID").is_in(base_dispo["ID"])
        ).select([pl.col("ID"), pl.col(columna_fc).alias(columna_fc)]),
        on="ID",
        how="left"
    )
else:
    print(f"Columna {columna_fc} no encontrada en fc_pvt.")

In [156]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24),Sem 49 (24)_tr,Sem 49 (24)_fc
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64,i64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0,2592,187.066975
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,-144.0,2592,187.066975
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,-24.0,2592,187.066975
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,134.715935,768,69.284065
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,99.621247,768,92.378753
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null,0.0
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null,0.0
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""",

In [157]:
columna_nombre_siguiente

'Sem 48 (24)'

## Creando Sem3 
1. **Crea una nueva columna** llamada `Sem {semana_actual + 2} ({año_actual})` y la inicializa con valores `None` (nulos).
2. **Sobrescribe esa columna** con los valores de `columna_nombre_siguiente`, copiando todos los valores (positivos, negativos y nulos) sin aplicar filtros ni modificaciones.


In [158]:
base_dispo = base_dispo.with_columns(
    pl.lit(None).alias(f"Sem {semana_actual + 2} ({año_actual})")
)

base_dispo = base_dispo.with_columns(
    pl.col(columna_nombre_siguiente).alias(f"Sem {semana_actual + 2} ({año_actual})")
)


In [159]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24),Sem 49 (24)_tr,Sem 49 (24)_fc,Sem 49 (24)
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64,i64,f64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0,2592,187.066975,-48.0
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,-144.0,2592,187.066975,-144.0
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,-24.0,2592,187.066975,-24.0
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,134.715935,768,69.284065,134.715935
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,99.621247,768,92.378753,99.621247
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null,0.0,0.0
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,null,0.0,0.0
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RE

## Calculo fundamental de Sem3 
1. **Suma Inicial:**
   - Actualiza la columna `sem_tres` sumando:
     - Su valor original.
     - El producto ponderado de `f"{sem_tres}_tr"`.
     - El producto ponderado de la columna `"711"`.
   - Maneja valores nulos reemplazándolos con 0.

2. **Evaluación y Ajuste:**
   - Si el valor actualizado de `sem_tres` es mayor a 0:
     - Se resta el valor de `f"{sem_tres}_fc`.
     - Si el resultado es positivo o cero, se conserva.
     - Si el resultado es negativo, se asigna 0.
   - Si el valor de `sem_tres` no es mayor a 0, se deja sin modificar, conservando la suma inicial.


In [160]:
# Suma inicial de sem_tres, tr ponderado y "711" ponderado
base_dispo = base_dispo.with_columns(
    (
        pl.col(sem_tres) + 
        (pl.col(f"{sem_tres}_tr") * pl.col("Ponderación por canal")).fill_null(0) + 
        (pl.col("711") * pl.col("Ponderación por canal")).fill_null(0)
    )
    .alias(sem_tres)
)

# Evaluación y ajuste de sem_tres
base_dispo = base_dispo.with_columns(
    pl.when(
        pl.col(sem_tres) > 0  # Verifica si el resultado es mayor a 0
    )
    .then(
        pl.when(
            (pl.col(sem_tres) - pl.col(f"{sem_tres}_fc").fill_null(0)) >= 0  # Si la resta no es negativa
        )
        .then(
            pl.col(sem_tres) - pl.col(f"{sem_tres}_fc").fill_null(0)  # Mantiene el resultado de la resta
        )
        .otherwise(0)  # Si la resta es negativa, asigna 0
    )
    .otherwise(pl.col(sem_tres))  # Si la condición no se cumple, deja solo la suma inicial
)
base_dispo = base_dispo.drop([columna_tr, columna_fc])


In [161]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24),Sem 49 (24)
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0,2356.933025
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,-144.0,2260.933025
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,-24.0,2380.933025
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,134.715935,833.431871
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""450""","""0.416666667""",0.0,210.0,0.0,303.0,18,156.0,141,192.0,99.621247,775.242494
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1171967CL RETAIL""","""1171967""","""LIMP.ESPEJO TELESCÓPICO ML AP2…","""MOTORLIFE""",0.0,"""0""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""CUIDADO Y MANTENCION""","""PARABRISAS Y ACCESORIOS""","""PLUMILLAS""","""Otros""","""17968""","""WICO INTERNATION LIMITED""","""IMP""","""158""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
"""151798CL RETAIL""","""151798""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-1600""","""Implementos""","""7655""","""MINOS DIS TICARET LIMITED""","""IMP""","""150""","""NO""","""Sin Necesidad""",null,"""0""","""0""","""SIN SEGMENTACION""","""0.166666667""","""0""",0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
"""151799CL RETAIL""","""151799""",null,null,0.0,"""1""","""CL RETAIL""","""RETAIL""","""Rep.Alter.Maquinaria""","""IMPLEMENTOS""","""POST COSECHA""","""TRITURADORES""","""MODELO DSP-

In [162]:
base_dispo2=base_dispo

In [163]:
base_dispo=base_dispo2

In [164]:
print(base_dispo[3])

shape: (1, 36)
┌─────────────┬─────────┬─────────────┬───────┬───┬─────────────┬────────┬────────────┬────────────┐
│ ID          ┆ UE      ┆ Descripcion ┆ Marca ┆ … ┆ Stock       ┆ Sem 47 ┆ Sem 48     ┆ Sem 49     │
│ ---         ┆ ---     ┆ Material    ┆ ---   ┆   ┆ Objetivo    ┆ (24)   ┆ (24)       ┆ (24)       │
│ str         ┆ str     ┆ ---         ┆ str   ┆   ┆ ---         ┆ ---    ┆ ---        ┆ ---        │
│             ┆         ┆ str         ┆       ┆   ┆ i64         ┆ f64    ┆ f64        ┆ f64        │
╞═════════════╪═════════╪═════════════╪═══════╪═══╪═════════════╪════════╪════════════╪════════════╡
│ 1171298CL   ┆ 1171298 ┆ SCENT BOMB  ┆ SCENT ┆ … ┆ 122         ┆ 204.0  ┆ 134.715935 ┆ 833.431871 │
│ RETAIL      ┆         ┆ VENT CLIP   ┆ BOMB  ┆   ┆             ┆        ┆            ┆            │
│             ┆         ┆ MANG TROP…  ┆       ┆   ┆             ┆        ┆            ┆            │
└─────────────┴─────────┴─────────────┴───────┴───┴─────────────┴────────┴──

## BUCLE DESDE LA CUARTA SEMANA :D

1. **Obtención de la fecha y semana actual**: 
   - Se determina la semana actual y se ajusta para considerar el cambio de año cuando sea necesario.

2. **Iteración por semanas futuras**: 
   - Se proyectan las próximas 50 semanas, calculando los nombres de las semanas actuales y anteriores.

3. **Verificación y unión de datos**: 
   - Se verifica la existencia de columnas en los datos externos (`transito_pvt` y `fc_pvt`).
   - Si las columnas no existen, se crean con valores predeterminados (0).

4. **Cálculo de valores por semana**: 
   - Usa datos de semanas anteriores, factores de ponderación y valores de forecast para calcular los valores proyectados de la semana actual.

5. **Limpieza de columnas temporales**: 
   - Se eliminan las columnas usadas para el cálculo, manteniendo solo las necesarias.

6. **Gestión de errores**: 
   - Se implementa un manejo de errores por iteración, registrando cualquier problema.


In [165]:
import datetime
import polars as pl

# Obtener la semana y año actual
fecha_actual = datetime.date.today()
semana_actual = fecha_actual.isocalendar()[1]
año_actual = int(str(fecha_actual.year)[-2:])

# Número total de semanas en un año ISO
SEMANAS_POR_AÑO = 52

# Iterar por 50 semanas ajustando el cruce de años
for i in range(1, 51):  # 50 semanas a partir de la actual
    try:
        semana_iteracion = semana_actual + i
        if semana_iteracion > SEMANAS_POR_AÑO:
            semana_iteracion -= SEMANAS_POR_AÑO
            año_iteracion = str(año_actual + 1).zfill(2)
        else:
            año_iteracion = str(año_actual).zfill(2)

        # Calcular el nombre de la semana anterior
        if semana_iteracion == 1:  # Primera semana del nuevo año
            semana_anterior = f"Sem {SEMANAS_POR_AÑO} ({str(int(año_iteracion) - 1).zfill(2)})"
        else:
            semana_anterior = f"Sem {semana_iteracion - 1} ({año_iteracion})"

        # Nombres de las columnas para la semana actual
        semana_actual_nombre = f"Sem {semana_iteracion} ({año_iteracion})"
        columna_tr = f"{semana_actual_nombre}_tr"
        columna_fc = f"{semana_actual_nombre}_fc"

        # Inicializar las columnas que serán usadas
        columnas_usadas = []

        # Verificar y agregar columna desde `transito_pvt` si existe
        if columna_tr in transito_pvt.columns:
            base_dispo = base_dispo.join(
                transito_pvt.filter(
                    pl.col("UE").is_in(base_dispo["UE"])
                ).select([pl.col("UE"), pl.col(columna_tr).alias(columna_tr)]),
                on="UE",
                how="left"
            )
            columnas_usadas.append(columna_tr)
        else:
            base_dispo = base_dispo.with_columns(
                pl.lit(0).alias(columna_tr)  # Crear columna con valor 0 si no existe
            )
            columnas_usadas.append(f"{columna_tr} (asignado 0)")

        # Verificar y agregar columna desde `fc_pvt` si existe
        if columna_fc in fc_pvt.columns:
            base_dispo = base_dispo.join(
                fc_pvt.filter(
                    pl.col("ID").is_in(base_dispo["ID"])
                ).select([pl.col("ID"), pl.col(columna_fc).alias(columna_fc)]),
                on="ID",
                how="left"
            )
            columnas_usadas.append(columna_fc)
        else:
            base_dispo = base_dispo.with_columns(
                pl.lit(0).alias(columna_fc)  # Crear columna con valor 0 si no existe
            )
            columnas_usadas.append(f"{columna_fc} (asignado 0)")

        # Calcular los valores para la semana actual con manejo de nulos
        base_dispo = base_dispo.with_columns(
            pl.when(
                (pl.col(semana_anterior).fill_null(0) +
                 (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1))) > 0
            )
            .then(
                (pl.col(semana_anterior).fill_null(0) +
                 (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1)) -
                 pl.col(columna_fc).fill_null(0))
            )
            .otherwise(
                pl.col(semana_anterior).fill_null(0) +
                (pl.col(columna_tr).fill_null(0) * pl.col("Ponderación por canal").fill_null(1))
            )
            .alias(semana_actual_nombre)
        )

        # Eliminar columnas temporales
        base_dispo = base_dispo.drop([columna_tr, columna_fc])

        # Mostrar qué columnas fueron usadas en esta iteración
        print(f"Iteración {semana_actual_nombre}: Se usaron columnas: {', '.join(columnas_usadas)}")

    except Exception as e:
        print(f"Error en la iteración {semana_actual_nombre}: {e}")


Iteración Sem 48 (24): Se usaron columnas: Sem 48 (24)_tr, Sem 48 (24)_fc
Iteración Sem 49 (24): Se usaron columnas: Sem 49 (24)_tr, Sem 49 (24)_fc
Iteración Sem 50 (24): Se usaron columnas: Sem 50 (24)_tr, Sem 50 (24)_fc
Iteración Sem 51 (24): Se usaron columnas: Sem 51 (24)_tr, Sem 51 (24)_fc
Iteración Sem 52 (24): Se usaron columnas: Sem 52 (24)_tr, Sem 52 (24)_fc
Iteración Sem 1 (25): Se usaron columnas: Sem 1 (25)_tr, Sem 1 (25)_fc
Iteración Sem 2 (25): Se usaron columnas: Sem 2 (25)_tr, Sem 2 (25)_fc
Iteración Sem 3 (25): Se usaron columnas: Sem 3 (25)_tr, Sem 3 (25)_fc
Iteración Sem 4 (25): Se usaron columnas: Sem 4 (25)_tr, Sem 4 (25)_fc
Iteración Sem 5 (25): Se usaron columnas: Sem 5 (25)_tr, Sem 5 (25)_fc
Iteración Sem 6 (25): Se usaron columnas: Sem 6 (25)_tr, Sem 6 (25)_fc
Iteración Sem 7 (25): Se usaron columnas: Sem 7 (25)_tr, Sem 7 (25)_fc
Iteración Sem 8 (25): Se usaron columnas: Sem 8 (25)_tr, Sem 8 (25)_fc
Iteración Sem 9 (25): Se usaron columnas: Sem 9 (25)_tr, Sem 9

# Proceso de Cálculo de Nuevas Columnas para saber el % de Dispo :D!

1. **Filtrado de Columnas:**
   - Seleccionar columnas que:
     - Comienzan con `"Sem"`.
     - Contienen `"("` en su nombre.
     - No contienen `"D"` en su nombre.

2. **Nuevas Columnas:**
   - Crear una nueva columna con el mismo nombre que la base, añadiendo `"D"` al final.

3. **Reglas de Cálculo:**
   - **Regla 1:** Si el valor es negativo → Asignar `0`.
   - **Regla 2:** Si el valor es `0` y forecast (`columna_fc`) > 0 → Asignar `0`.
   - **Regla 3:** Si el valor es `0` y forecast no existe o es `0` → Asignar `1`.
   - **Regla 4:** Si el valor es mayor que `0` y forecast no existe o es `0` → Asignar `1`.
   - **Regla 5:** Si el valor es mayor que `0` y forecast existe:
     - Calcular valor / forecast.
     - Si el resultado > 1 → Asignar `1`.
     - Si el resultado ≤ 1 → Asignar el resultado.


In [ ]:
for columna in base_dispo.columns:
    if columna.startswith("Sem") and "(" in columna and "D" not in columna:
        columna_fc = f"{columna}_fc"  # Nombre de la columna_fc correspondiente
        nueva_columna = f"{columna}D"  # Nombre de la nueva columna

        # Verificar y agregar columna_fc desde `fc_pvt` si existe
        if columna_fc in fc_pvt.columns:
            base_dispo = base_dispo.join(
                fc_pvt.filter(
                    pl.col("ID").is_in(base_dispo["ID"])
                ).select([pl.col("ID"), pl.col(columna_fc).alias(columna_fc)]),
                on="ID",
                how="left"
            )
        else:
            # Crear columna_fc con valor 0 si no existe
            base_dispo = base_dispo.with_columns(
                pl.lit(0).alias(columna_fc)
            )

        # Crear la nueva columna basada en las condiciones especificadas
        base_dispo = base_dispo.with_columns(
            (
                pl.when(pl.col(columna) < 0)  # Condición inicial: Sem < 0
                .then(0)
                .when((pl.col(columna) == 0) & (pl.col(columna_fc) > 0))  # Segunda condición
                .then(0)
                .when((pl.col(columna) == 0) & ((pl.col(columna_fc) == 0) | pl.col(columna_fc).is_null()))  # Tercera condición
                .then(1)
                .when((pl.col(columna) > 0) & ((pl.col(columna_fc) == 0) | pl.col(columna_fc).is_null()))  # Cuarta condición
                .then(1)
                .when((pl.col(columna) > 0) & (pl.col(columna_fc) > 0))  # Quinta condición
                .then(
                    pl.when(pl.col(columna) / pl.col(columna_fc) > 1)
                    .then(1)
                    .otherwise(pl.col(columna) / pl.col(columna_fc))
                )
                * 100  # Multiplicar todos los resultados por 100
            ).alias(nueva_columna)
        )

        # Eliminar la columna_fc después de la iteración
        base_dispo = base_dispo.drop(columna_fc)


In [167]:
base_dispo

ID,UE,Descripcion Material,Marca,Ponderación por canal,Vig,Canal,Canal Consolidado,Sector,Agrupación ClaCom,SubAgrupación ClaCom,Categoría ClaCom,SubCategoría ClaCom,Bobinas/Implementos/Otros,COD. PROVEEDOR,PROVEEDOR,ORIGEN,LEAD TIME,Codigos Transparentes,Comentario Sesión,Comentario Finales,OBS Retail,OBS DERCO,Segmentacion AFM,Prom Forecast 3-6-12,Prom Venta 3-6-12,711,Stock CD,Control,Stock Tiendas,Faltante,Sobrante,Stock Objetivo,Sem 47 (24),Sem 48 (24),Sem 49 (24),Sem 50 (24),…,Sem 9 (25)D,Sem 10 (25)D,Sem 11 (25)D,Sem 12 (25)D,Sem 13 (25)D,Sem 14 (25)D,Sem 15 (25)D,Sem 16 (25)D,Sem 17 (25)D,Sem 18 (25)D,Sem 19 (25)D,Sem 20 (25)D,Sem 21 (25)D,Sem 22 (25)D,Sem 23 (25)D,Sem 24 (25)D,Sem 25 (25)D,Sem 26 (25)D,Sem 27 (25)D,Sem 28 (25)D,Sem 29 (25)D,Sem 30 (25)D,Sem 31 (25)D,Sem 32 (25)D,Sem 33 (25)D,Sem 34 (25)D,Sem 35 (25)D,Sem 36 (25)D,Sem 37 (25)D,Sem 38 (25)D,Sem 39 (25)D,Sem 40 (25)D,Sem 41 (25)D,Sem 42 (25)D,Sem 43 (25)D,Sem 44 (25)D,Sem 45 (25)D
str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1171331CL RETAIL""","""1171331""","""SCENT BOMB 2 PACK PAPER MAN TR…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.25""",0.0,0.0,0.0,1446.0,48,1133.0,292,-48.0,-48.0,2356.933025,2169.866051,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.307852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1171332CL RETAIL""","""1171332""","""SCENT BOMB 2 PACK PAPER HAWA B…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""0.5""",0.0,0.0,0.0,1276.0,144,985.0,292,-144.0,-144.0,2260.933025,2073.866051,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.794667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1171330CL RETAIL""","""1171330""","""SCENT BOMB 2 PACK PAPER B CHRY…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""COLGANTE""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""911.333333333""","""1""",0.0,0.0,0.0,1417.0,24,1103.0,292,-24.0,-24.0,2380.933025,2193.866051,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.436148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1171298CL RETAIL""","""1171298""","""SCENT BOMB VENT CLIP MANG TROP…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""","""61111""","""Careco LLC""","""IMP""","""120""","""NO""","""No Comprar""",null,"""0""","""0""","""SIN SEGMENTACION""","""350""","""0.416666667""",0.0,222.0,0.0,293.998,18,167.998,122,204.0,134.715935,833.431871,764.147806,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.7316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""1171325CL RETAIL""","""1171325""","""SCENT BOMB VENT CLIP BLACK BOM…","""SCENT BOMB""",1.0,"""1""","""CL RETAIL""","""RETAIL""","""Accesorios-Car Care""","""ACCESORIOS""","""LIMPIEZA Y CUIDADO""","""AROMATIZANTES""","""VENTILACION""","""Otros""",""

Guardar archivo :DDD

In [168]:
import os
from datetime import datetime
import polars as pl

# Definir la ruta base
ruta_base = os.path.expanduser(r"~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Disponibilidad Futura\Dispo_Canales")

# Obtener el año y semana actual
anio_actual = datetime.now().year
semana_actual = datetime.now().isocalendar()[1]

# Crear la carpeta del año actual si no existe
ruta_anio = os.path.join(ruta_base, str(anio_actual))
os.makedirs(ruta_anio, exist_ok=True)

# Crear la carpeta de la semana actual dentro de la carpeta del año
ruta_semana = os.path.join(ruta_anio, f"Sem {semana_actual}")
os.makedirs(ruta_semana, exist_ok=True)

# Rutas para guardar los archivos
ruta_base_dispo = os.path.join(ruta_semana, "base_dispo.xlsx")
ruta_fc_pvt = os.path.join(ruta_semana, "fc_pvt.xlsx")
ruta_transito_pvt = os.path.join(ruta_semana, "transito_pvt.xlsx")

# Guardar los DataFrames en Excel sin aplicar formato
base_dispo.write_excel(ruta_base_dispo)
fc_pvt.write_excel(ruta_fc_pvt)
transito_pvt.write_excel(ruta_transito_pvt)

# Mensaje de confirmación
print(f"Archivos guardados exitosamente en la carpeta: {ruta_semana}")


Archivos guardados exitosamente en la carpeta: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\KPI\Disponibilidad Futura\Dispo_Canales\2024\Sem 47
